Here we test the idea that product manifolds can be built from Riemannian metrics. We have two approaches:
1. Train a neural network, generate a Riemann metric $g$ in the output space, and pull it back to the input space
2. Use the following equation with $\mathbf{x}$ as your dataset,
$$
\frac{\text{d}}{\text{d}t} \mathbf{x} = -g^{inv} \nabla Tr(g(x))
$$
for $T_max$ time steps. This should lead to convergence along the decision boundary.
4. Seperability is determined by the maximum margin; the number of time steps require for Euclidean $\varepsilon$ converge, i.e.
$$
||\mathbf{x}_{t+1} - \mathbf{x}_t||^2_2 < \varepsilon.
$$

OR

4. The seperability is determined by the 

In [1]:
import sys
sys.path.append("../../../")

import torch
import numpy as np
import os
import tqdm

from riemannian_geometry.computations.pullback_metric import pullback_all_metrics, pullback_ricci_tensor
from models.supervised.mlp.model import MLP
from models.data.sklearn_datasets import MoonDataset, SpiralDataset, BlobsDataset, CirclesDataset

models_path = "../../../models/supervised/mlp/saved_models"


In [2]:
np.random.seed(2)
torch.manual_seed(2)

In [3]:
mode = 'moon'
size = "skinny"
epoch = 199

if mode == 'moon':
    dataset = MoonDataset(n_samples=1000, noise=0.01)
elif mode == 'blobs':
    dataset = BlobsDataset(n_samples=1000, noise=0.01)
elif mode == 'spiral':
    dataset = SpiralDataset(n_samples=1000, noise=0.01)
elif mode == 'circles':
    dataset = CirclesDataset(n_samples=1000, noise=0.01)

if size == "skinny":
    model = MLP(2,7,2,2)
    full_path = f'{models_path}/2_wide/mlp_{mode}/model_{epoch}.pth'
elif size == "overfit":
    model = MLP(2,7,2,1)
    full_path = f'{models_path}/overfit/mlp_{mode}/model_{epoch}.pth'
else:
    model = MLP(2,7,10,2)
    full_path = f'{models_path}/vanilla/mlp_{mode}/model_{epoch}.pth'
model.load_state_dict(torch.load(full_path))
if size == "overfit":
    model.layers = model.layers[:-1]
    model.num_layers = len(model.layers)

In [4]:
N=50
wrt = "output_wise"
sigma = 0.05

X = torch.from_numpy(dataset.X).float()
labels = dataset.y


model.forward(X, save_activations=True)

activations = model.get_activations()
activations_np = [a.detach().numpy() for a in activations]
g, dg, ddg, surface = pullback_all_metrics(model, activations, N, wrt=wrt, method="manifold", sigma=sigma, normalised=True)
    

Rejected 249 points


In [8]:
g = np.random.rand(2,2)
dg = np.random.randn(2,2,2)
tr_dg = np.trace(dg, axis1=1, axis2=2)


[[0.98378659 0.50388998]
 [0.68324005 0.01895585]] [[[-0.88946763 -0.53716511]
  [ 0.45545618  0.1386256 ]]

 [[-0.21024178 -1.32743963]
  [ 0.32692527  1.62574608]]] [-0.75084203  1.4155043 ]


In [6]:
def flow_solution(g, surface, T_max, delta_t):
    """
    g: metric tensor defined at each point on the surface
    dg: derivative of the metric tensor defined at each point on the surface
    input_points: surface of points
    T_max: maximum time
    delta_t: step size
    """
    reference_frame = surface.copy()
    g_inv = np.linalg.inv(g)
    tr_g = np.trace(g, axis1=1, axis2=2)

    def _update_step(points, g_inv, dg, delta_t):
        return points + delta_t * np.einsum('nij,nj->ni', g_inv, dg)
    
    def _interpolate(reference_points, points):
        

    for t in range(T_max):
    return points

8
